In [ ]:


import os
import random
import SimpleITK as sitk
import numpy as np

def read_random_lesion_segmentation(brats_data_dir):
    # Get a list of patient directories in the BRATS dataset
    patient_dirs = [d for d in os.listdir(brats_data_dir) if os.path.isdir(os.path.join(brats_data_dir, d))]

    # Randomly select a patient directory
    random_patient_dir = random.choice(patient_dirs)

    # Get the segmentation file path (assuming the segmentation file is in the "seg" subdirectory)
    segmentation_dir = os.path.join(brats_data_dir, random_patient_dir, 'seg')
    segmentation_files = [f for f in os.listdir(segmentation_dir) if f.endswith('.nii.gz')]
    
    if not segmentation_files:
        print(f"No segmentation files found for patient {random_patient_dir}")
        return None

    random_segmentation_file = random.choice(segmentation_files)
    segmentation_file_path = os.path.join(segmentation_dir, random_segmentation_file)

    # Read the segmentation using SimpleITK
    segmentation_image = sitk.ReadImage(segmentation_file_path)
    
    # Convert the segmentation image to a NumPy array
    segmentation_array = sitk.GetArrayFromImage(segmentation_image)

    return segmentation_array

# Example usage:
brats_data_directory = '/path/to/brats/dataset'
random_lesion_segmentation = read_random_lesion_segmentation(brats_data_directory)

if random_lesion_segmentation is not None:
    # Process or visualize the segmentation as needed
    print(f"Random lesion segmentation shape: {random_lesion_segmentation.shape}")


In [ ]:
from nilearn.plotting import plot_anat


# load random mask from BRATS

# Load normal image

# make pre lesion as lesion core

# make post lesion as full lesion

# plot

pre_lesion = '/deneb_disk/auto_resection/lesion_masks/lesion_init.mask.nii.gz'
post_lesion = '/deneb_disk/auto_resection/lesion_masks/lesion11.mask.nii.gz'

cross_hair = (56, 105, 122)
plot_anat(pre_lesion, title='Pre-lesion mask', cut_coords=cross_hair)
plot_anat(post_lesion, title='Post-lesion mask', cut_coords=cross_hair)


In [ ]:

from warper_incompressible import Warper

nonlin_reg = Warper()
pre2post_lesion = '/deneb_disk/auto_resection/lesion_masks/pre2post_lesion_incompressible.nii.gz'
ddf = '/deneb_disk/auto_resection/lesion_masks/pre2post_lesion_ddf_incompressible.nii.gz'
jac_file = '/deneb_disk/auto_resection/lesion_masks/pre2post_lesion_jac_incompressible.nii.gz'
nonlin_reg.nonlinear_reg(
    target_file=post_lesion,
    moving_file=pre_lesion,
    output_file=pre2post_lesion,
    ddf_file=ddf,
    reg_penalty=1e-3,
    nn_input_size=64,
    lr=1e-3,
    max_epochs=3000,
    loss="mse",
    jacobian_determinant_file=jac_file)


In [ ]:
from warp_utils import apply_warp
from monai.transforms import LoadImage, EnsureChannelFirst
import nibabel as nib

mri = '/deneb_disk/auto_resection/lesion_masks/USCBrain.nii.gz'
plot_anat(mri, title='MRI', cut_coords=cross_hair)

plot_anat(pre2post_lesion, title='Pre2post lesion mask', cut_coords=cross_hair)

moving = LoadImage(image_only=True)(pre_lesion)
moving = EnsureChannelFirst()(moving)[None]
target = LoadImage(image_only=True)(post_lesion)
target = EnsureChannelFirst()(target)[None]
disp = LoadImage(image_only=True)(ddf)
disp = EnsureChannelFirst()(disp)[None]
mri = LoadImage(image_only=True)(mri)
mri = EnsureChannelFirst()(mri)[None]

output_file='/deneb_disk/auto_resection/lesion_masks/pre2post_lesion_check_incompressible.nii.gz'
moved = apply_warp(disp_field=disp, moving_image=moving, target_image=target)
nib.save(nib.Nifti1Image(moved[0, 0].detach().cpu().numpy(), target.affine), output_file)


target = LoadImage(image_only=True)(post_lesion)
target = EnsureChannelFirst()(target)[None]
disp = LoadImage(image_only=True)(ddf)
disp = EnsureChannelFirst()(disp)[None]
#mri = LoadImage(image_only=True)(mri)
#mri = EnsureChannelFirst()(mri)[None]
output_file='/deneb_disk/auto_resection/lesion_masks/pre2post_mri_incompressible.nii.gz'
moved = apply_warp(disp_field=disp, moving_image=mri, target_image=target)
nib.save(nib.Nifti1Image(moved[0, 0].detach().cpu().numpy(), target.affine), output_file)


In [ ]:

lesion_file = '/deneb_disk/auto_resection/lesion_masks/lesion_init0.mask.nii.gz'
lesion_mri = '/deneb_disk/auto_resection/lesion_masks/USCBrain.pre_lesion.nii.gz'

target = LoadImage(image_only=True)(post_lesion)
target = EnsureChannelFirst()(target)[None]
disp = LoadImage(image_only=True)(ddf)
disp = EnsureChannelFirst()(disp)[None]
lesion = LoadImage(image_only=True)(lesion_file)
lesion = EnsureChannelFirst()(lesion)[None]
lesion = mri + lesion
nib.save(nib.Nifti1Image(lesion[0, 0].detach().cpu().numpy(), target.affine), lesion_mri)

output_file='/deneb_disk/auto_resection/lesion_masks/pre2post_lesion_incompressible.nii.gz'
moved = apply_warp(disp_field=disp, moving_image=lesion, target_image=target)
nib.save(nib.Nifti1Image(moved[0, 0].detach().cpu().numpy(), target.affine), output_file)


plot_anat(lesion_mri, title='Pre lesion mask', cut_coords=cross_hair)
plot_anat(output_file, title='Pre2post lesion mask', cut_coords=cross_hair)
